# Quantium Virtual Internship - Retail Strategy and Analytics - Task 1

## Loading required libraries

In [161]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline

## Loading required datasets

In [ ]:
# getting present working directory
pwd = os.getcwd()

In [ ]:
transaction_data = pd.read_excel(pwd + '\\QVI_transaction_data.xlsx')

In [ ]:
customer_data = pd.read_csv(pwd + '\\QVI_purchase_behaviour.csv')

## Exploratory data analysis

### Examining transaction data

In [ ]:
transaction_data

Let's check if columns we would expect to be numeric are in numeric form and date columns are in date format. 

In [ ]:
transaction_data.info()

We can see that the date column is in an integer format. Let's change this to a date format.

In [ ]:
base_date = pd.Timestamp('30-12-1899')
transaction_data.DATE = transaction_data.DATE.apply(lambda x: base_date + pd.DateOffset(x))

We should check that we are looking at the right products by examining PROD_NAME.

In [ ]:
transaction_data.PROD_NAME.unique()

Looks like we are definitely looking at potato chips but how can we check that these are all chips? We can do some basic text analysis by summarising the individual words in the product name

In [ ]:
# Making a list of all the words in product names
product_words = []
def add_words(lst):
    for y in lst:
        product_words.append(y)
    return
product_words1 = transaction_data.PROD_NAME.apply(lambda x: add_words(x.split()))
product_words

As we are only interested in words that will tell us if the product is chips or not, let's remove all words with digits and special characters such as '&' from our set of product words

In [ ]:
# Removing digits
regex1 = re.compile(r'^.+\d.+$')
product_words = [i for i in product_words if not regex1.match(i)]
# Removing special characters
regex2 = re.compile(r'^.+\W.+$')
regex3 = re.compile(r'\W')
product_words = [i for i in product_words if not regex1.match(i)]
product_words = [i for i in product_words if not regex2.match(i)]
product_words = [i for i in product_words if not regex3.match(i)]
product_words = pd.DataFrame(product_words)
product_words.value_counts()

There are salsa products in the dataset but we are only interested in the chips category, so let's remove these. 

In [ ]:
product_words_list = transaction_data.PROD_NAME.apply(lambda x: 'Salsa' not in x.split())
transaction_data = transaction_data[product_words_list == True]
transaction_data

Next, we can use `describe()` to check summary statistics such as mean, min and max values for each feature to see if there are any obvious outliers in the data and if there are any nulls in any of the columns.

In [ ]:
transaction_data.describe()

There are no nulls in the columns but product quantity appears to have an outlier which we should investigate further. Let's investigate further the case where 200 packets of chips are bought in one transaction.


In [ ]:
transaction_data[transaction_data.PROD_QTY == 200]

There are two transactions where 200 packets of chips are bought in one transaction and both of these transactions were by the same customer.

Let's see if the customer has had other transactions

In [ ]:
transaction_data[transaction_data.LYLTY_CARD_NBR == 226000]

It looks like this customer has only had the two transactions over the year and is not an ordinary retail customer. The customer might be buying chips for commercial purposes instead. We'll remove this loyalty card number from further analysis.

In [ ]:
transaction_data = transaction_data.loc[transaction_data.LYLTY_CARD_NBR != 226000]
# Re-examining transaction data
transaction_data[transaction_data.LYLTY_CARD_NBR == 226000]

That's better. Now, let's look at the number of transaction lines over time to see if there are any obvious data issues such as missing data.

In [ ]:
txn_count = transaction_data.groupby('DATE').count().TXN_ID.reset_index()

There's only 364 rows, meaning only 364 dates which indicates a missing date. Let's create a sequence of dates from 1 Jul 2018 to 30 Jun 2019 and use this to create a chart of number of transactions over time to find the missing date.

In [ ]:
date_list = pd.date_range(start="2018-07-01",end="2019-06-30")
date_list = pd.DataFrame(date_list)
date_list.rename(columns={0: 'DATE'}, inplace=True)
merged_txn_count = pd.merge(left=date_list, right=txn_count, how='left', on='DATE')
merged_txn_count.rename(columns={'TXN_ID': 'TXN_COUNT'}, inplace=True)
merged_txn_count

In [ ]:
plt.figure(figsize=(16,9))
plt.xticks(rotation='vertical')
plt.title('Transactions over time', fontdict={'fontsize': 20})
plt.xlabel('Day')
plt.ylabel('Number of transactions')
plt.bar(merged_txn_count.DATE, merged_txn_count.TXN_COUNT, width=0.4, color='b');

We can see that there is an increase in purchases in December and a break in late December. Let's zoom in on this.

In [ ]:
december_data = merged_txn_count.loc[range(153, 184)]
plt.figure(figsize=(11,8))
plt.xticks(rotation='vertical')
plt.title('Transactions over December', fontdict={'fontsize': 20})
plt.xlabel('Day', fontdict={'fontsize': 15})
plt.ylabel('Number of transactions', fontdict={'fontsize': 15})
plt.bar(december_data.DATE, december_data.TXN_COUNT, width=0.5);

We can see that the increase in sales occurs in the lead-up to Christmas and that there are zero sales on Christmas day itself. This is due to shops being closed on Christmas day.

Now that we are satisfied that the data no longer has outliers, we can move on to creating other features such as brand of chips or pack size from PROD_NAME. We will start with pack size

In [ ]:
transaction_data['PACKET_SIZE'] = transaction_data.PROD_NAME.apply(lambda x: int(re.search(r'\d+', x).group()))
transaction_data.sort_values(by='PACKET_SIZE')

The largest size is 380g and the smallest size is 70g - seems sensible!

Let's plot a histogram of PACK_SIZE since we know that it is a categorical variable and not a continuous variable even though it is numeric

In [ ]:
plt.figure(figsize=(11,8))
plt.title('Counts of Packet Sizes', fontdict={'fontsize': 20})
plt.xlabel('Packet Size', fontdict={'fontsize': 15})
plt.ylabel('Count', fontdict={'fontsize': 15})
sns.histplot(data=transaction_data, x='PACKET_SIZE', binwidth = 10);

Pack sizes created look reasonable.

Now to create brands, we can use the first word in PROD_NAME to work out the brand name.

In [ ]:
transaction_data['BRAND'] = transaction_data.PROD_NAME.apply(lambda x: x.split()[0])
transaction_data

In [ ]:
transaction_data.BRAND.unique()

Some of the brand names look like they are of the same brands - such as RED and RRD, which are both Red Rock Deli chips. Let's combine these together and also add their full name. We'll also do the same for others where its required.

In [ ]:
transaction_data.loc[transaction_data.BRAND == 'Natural', 'BRAND'] = 'Natural Chip Company'
transaction_data.loc[transaction_data.BRAND == 'NCC', 'BRAND'] = 'Natural Chip Company'
transaction_data.loc[transaction_data.BRAND == 'Old', 'BRAND'] = 'Old El Paso'
transaction_data.loc[transaction_data.BRAND == 'Burger', 'BRAND'] = 'Burger Rings'
transaction_data.loc[transaction_data.BRAND == 'Red', 'BRAND'] = 'Red Rock Deli'
transaction_data.loc[transaction_data.BRAND == 'RRD', 'BRAND'] = 'Red Rock Deli'
transaction_data.loc[transaction_data.BRAND == 'Grain', 'BRAND'] = 'Grain Waves'
transaction_data.loc[transaction_data.BRAND == 'Dorito', 'BRAND'] = 'Doritos'
transaction_data.loc[transaction_data.BRAND == 'Smith', 'BRAND'] = 'Smiths'
transaction_data.loc[transaction_data.BRAND == 'GrnWves', 'BRAND'] = 'Grain Waves'
transaction_data.loc[transaction_data.BRAND == 'Infzns', 'BRAND'] = 'Infuzions'
transaction_data.loc[transaction_data.BRAND == 'French', 'BRAND'] = 'French Fries'
transaction_data.loc[transaction_data.BRAND == 'Snbts', 'BRAND'] = 'Sunbites'
transaction_data.loc[transaction_data.BRAND == 'WW', 'BRAND'] = 'Woolworths'

Now let's create a price column for each product.

In [ ]:
transaction_data['PRICE'] = transaction_data.TOT_SALES / transaction_data.PROD_QTY

Let's reorder the column to our preference.

In [ ]:
transaction_data_cleaned = transaction_data[['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_NBR', 'PROD_NAME', 'BRAND', 'PACKET_SIZE', 'PROD_QTY', 'PRICE', 'TOT_SALES']]
transaction_data_cleaned

In [ ]:
#total_sales_df = pd.DataFrame(transaction_data_cleaned.groupby('DATE')['TOT_SALES'].sum()).reset_index()
#total_sales_df

Now that we are happy with the transaction dataset, let's have a look at the customer dataset.

## Examining customer data

In [ ]:
customer_data

In [ ]:
customer_data.info()

Seems like there are no Null values in the dataset which is great.

In [ ]:
customer_data.PREMIUM_CUSTOMER.value_counts()

Premium customers are less in number which is expected.

In [ ]:
customer_data.LIFESTAGE.value_counts()

Retirees, older singles/couples and young singles/couples are buying from the supermarket significantly more than the other groups.

## Merging transaction data to customer data

In [ ]:
merged_data = pd.merge(left=transaction_data_cleaned, right=customer_data, how='left', on='LYLTY_CARD_NBR')
merged_data

As the number of rows in `merged_data` is the same as that of `transaction_data`, we can be sure that no duplicates were created. This is because we created `merge_data` by setting `how='left'` (in other words, a left join) which means take all the rows in `transaction_data` and find rows with matching values in shared columns and then joining the details in these rows to the `LYLTY_CARD_NBR` or the first mentioned table.


Let's also check if some customers were not matched on by checking for nulls.

In [ ]:
merged_data.LYLTY_CARD_NBR.count()

Great, there are no nulls! So all our customers in the transaction data has been accounted for in the customer dataset.

Data exploration is now complete!

In [ ]:
#merged_data.to_csv('merged_data.csv', index=False)

## Data analysis on customer segments

Now that the data is ready for analysis, we can define some metrics of interest to the client:
- Who spends the most on chips (total sales), describing customers by lifestage and how premium their general purchasing behaviour is
- How many customers are in each segment
- How many chips are bought per customer by segment
- What's the average chip price by customer segment

We could also ask our data team for more information. Examples are:
- The customer's total spend over the period and total spend for each transaction to understand what proportion of their grocery spend is on chips
- Proportion of customers in each customer segment overall to compare against the mix of customers who purchase chips


Let's start with calculating total sales by LIFESTAGE and PREMIUM_CUSTOMER and plotting the split by these segments to describe which customer segment contribute most to chip sales.

In [ ]:
sales_data = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'])['TOT_SALES'].sum().to_frame().reset_index()
sales_data

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=sales_data, x='LIFESTAGE', y='TOT_SALES', hue='PREMIUM_CUSTOMER')
ax.set_xlabel('Customer Segment', fontsize=15) 
ax.set_ylabel('Total Sales', fontsize=15)
ax.set_title('Total Sales by Customer Segments', fontsize=20)
plt.xticks(rotation='vertical')
plt.show()

Sales are coming mainly from Budget - older families, Mainstream - young singles/couples, and Mainstream - retirees.

Let's see if the higher sales are due to there being more customers who buy chips.

In [ ]:
sales_data_customer = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'])['LYLTY_CARD_NBR'].count().to_frame().reset_index()
sales_data_customer

In [ ]:
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=sales_data_customer, x='LIFESTAGE', y='LYLTY_CARD_NBR', hue='PREMIUM_CUSTOMER')
ax.set_xlabel('Customer Segment', fontsize=15) 
ax.set_ylabel('Total Customers', fontsize=15)
ax.set_title('Total Customers by Customer Segments', fontsize=20)
plt.xticks(rotation='vertical')
plt.show()

There are more Mainstream - young singles/couples and Mainstream - retirees who buy chips. This contributes to there being more sales to these customer segments but this is not a major driver for the Budget - Older families segment.

Higher sales may also be driven by more units of chips being bought per customer. Let's have a look at this next.


In [ ]:
sales_data_avg = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'])['PROD_QTY'].mean().to_frame().reset_index()
sales_data_avg

In [ ]:
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=sales_data_avg, x='LIFESTAGE', y='PROD_QTY', hue='PREMIUM_CUSTOMER')
ax.set_xlabel('Customer Segment', fontsize=15) 
ax.set_ylabel('Average No. of Packets', fontsize=15)
ax.set_title('Average No. of Packets by Customer Segments', fontsize=20)
plt.legend(loc='lower right')
plt.xticks(rotation='vertical')
plt.show()

Older families and young families in general buy more chips per customer.

Let's also investigate the average price per unit chips bought for each customer segment as this is also a driver of total sales.

In [ ]:
sales_data_avg_price = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER'])['PRICE'].mean().to_frame().reset_index()
sales_data_avg_price

In [ ]:
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=sales_data_avg_price, x='LIFESTAGE', y='PRICE', hue='PREMIUM_CUSTOMER')
ax.set_xlabel('Customer Segment', fontsize=15) 
ax.set_ylabel('Average Price of Packets', fontsize=15)
ax.set_title('Average Price of Packets by Customer Segments', fontsize=20)
plt.legend(loc='lower right')
plt.xticks(rotation='vertical')
plt.show()

Mainstream midage and young singles and couples are more willing to pay more per packet of chips compared to their budget and premium counterparts. This may be due to premium shoppers being more likely to buy healthy snacks and when they buy chips, this is mainly for entertainment purposes rather than their own consumption. This is also supported by there being fewer premium midage and young singles and couples buying chips compared to their mainstream counterparts.

As the difference in average price per unit isn't large, we can check if this difference is statistically different.

In [ ]:
print(stats.ttest_ind(merged_data.loc[(merged_data.LIFESTAGE.isin(['MIDAGE SINGLES/COUPLES', 'YOUNG SINGLES/COUPLES'])) & (merged_data.PREMIUM_CUSTOMER == 'Mainstream')].PRICE,
                      merged_data.loc[(merged_data.LIFESTAGE.isin(['MIDAGE SINGLES/COUPLES', 'YOUNG SINGLES/COUPLES'])) & (merged_data.PREMIUM_CUSTOMER.isin(['Budget', 'Premium']))].PRICE),
stats.t.ppf(q=0.025, df=57365),
stats.t.ppf(q=0.975, df=57365))

The t-test results in a p-value of 2.235645611549355e-309, i.e. the unit price for mainstream, young and mid-age singles and couples are significantly higher than that of budget or premium, young and midage singles and couples.

## Deep dive into specific customer segments for insights

We have found quite a few interesting insights that we can dive deeper into.

We might want to target customer segments that contribute the most to sales to retain them or further increase sales. Let's look at Mainstream - young singles/couples. For instance, let's find out if they tend to buy a particular brand of chips.

In [ ]:
# mainstream_young_sin_coup = merged_data.loc[(merged_data.LIFESTAGE == 'YOUNG SINGLES/COUPLES') & (merged_data.PREMIUM_CUSTOMER == 'Mainstream')]
# mainstream_young_sin_coup_brand = mainstream_young_sin_coup.groupby('BRAND')['LYLTY_CARD_NBR'].count().to_frame().reset_index().sort_values(by='LYLTY_CARD_NBR', ascending=False)
# mainstream_young_sin_coup_brand.rename(columns={'LYLTY_CARD_NBR': 'CUSTOMER COUNT'}, inplace=True)

In [ ]:
# plt.figure(figsize=(16,9))
# ax = sns.barplot(data=mainstream_young_sin_coup_brand, x='BRAND', y='CUSTOMER COUNT')
# ax.set_xlabel('Product Name', fontsize=20) 
# ax.set_ylabel('No. of Customers', fontsize=20, labelpad=15)
# ax.set_title('No. of Customers by Brand', fontsize=25, pad=15)
# plt.xticks(rotation=70, fontsize=13)
# plt.yticks(fontsize=15)
# plt.show()

In [ ]:
# mainstream_young_sin_coup_pack = mainstream_young_sin_coup.groupby('PACKET_SIZE')['LYLTY_CARD_NBR'].count().to_frame().reset_index().sort_values(by='LYLTY_CARD_NBR', ascending=False)
# mainstream_young_sin_coup_pack.rename(columns={'LYLTY_CARD_NBR': 'CUSTOMER COUNT'}, inplace=True)

In [ ]:
# plt.figure(figsize=(16,9))
# ax = sns.barplot(data=mainstream_young_sin_coup_pack, x='PACKET_SIZE', y='CUSTOMER COUNT')
# ax.set_xlabel('Product Name', fontsize=20, labelpad=15) 
# ax.set_ylabel('Packet Size', fontsize=20, labelpad=15)
# ax.set_title('No. of Customers by Packet Size', fontsize=25, pad=15)
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.show()

### Preferred brand compared to the rest of the population

In [ ]:
cust_seg_brand = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER', 'BRAND'])['TOT_SALES'].sum().to_frame().reset_index()
cust_seg_brand

In [ ]:
max_cust_brand = cust_seg_brand.groupby('BRAND')['TOT_SALES'].max()
pop_seg_brand = cust_seg_brand.loc[cust_seg_brand['TOT_SALES'].isin(max_cust_brand)].sort_values(by='TOT_SALES', ascending=False).drop_duplicates(subset='BRAND')
pop_seg_brand.loc[(pop_seg_brand.LIFESTAGE == 'YOUNG SINGLES/COUPLES') & (pop_seg_brand.PREMIUM_CUSTOMER == 'Mainstream')]
#pop_seg_brand

These are the brands that the mainstream young singles and couples dominate over other population. This group should be targeted to retain them and to increase sales further.

### Preferred pack size compared to the rest of the population

In [ ]:
cust_seg_pack = merged_data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER', 'PACKET_SIZE'])['TOT_SALES'].sum().to_frame().reset_index()
cust_seg_pack

In [ ]:
max_cust_pack = cust_seg_pack.groupby('PACKET_SIZE')['TOT_SALES'].max()
pop_seg_pack = cust_seg_pack.loc[cust_seg_pack['TOT_SALES'].isin(max_cust_pack)].sort_values(by='TOT_SALES', ascending=False).drop_duplicates(subset='PACKET_SIZE')
pop_seg_pack.loc[(pop_seg_pack.LIFESTAGE == 'YOUNG SINGLES/COUPLES') & (pop_seg_pack.PREMIUM_CUSTOMER == 'Mainstream')]

The mainstream young singles and couples seem to buy more of the larger packet of chips over other population.